In [6]:
from sklearn.cluster import KMeans
import pandas as pd
import pickle
import string
import re
import string
from sklearn.model_selection import cross_validate as cross_validation, ShuffleSplit, cross_val_score, train_test_split, KFold
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import classification_report, accuracy_score, auc
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
import pickle

In [7]:
data_pre = pd.read_pickle('Cyclical_encoded_data.pkl')

In [46]:
def set_min_value(df):
    values = []
    for i in range(len(df.columns) - 2):
        values.append(-np.inf)
    values.insert(40,1) 
    values.pop()
    return values

def set_max_value(df):
    values = []
    for i in range(len(df.columns) - 2):
        values.append(np.inf)
    values.insert(40,4) 
    values.pop()
    return values 

In [47]:
print(data_pre.columns)
print(len(data_pre.columns))

Index(['YEAR', 'ITERATION', 'ID_HH', 'RHoMIS_ID', 'GPS_LAT', 'GPS_LON',
       'GPS_ALT', 'HHsizemembers', 'HHsizeMAE', 'LandOwned', 'LandCultivated',
       'LivestockHoldings', 'NrofMonthsFoodInsecure', 'PPI_Threshold',
       'PPI_Likelihood', 'score_HDDS_GoodSeason',
       'score_HDDS_farmbasedGoodSeason', 'score_HDDS_purchasedGoodSeason',
       'score_HDDS_BadSeason', 'score_HDDS_farmbasedBadSeason',
       'score_HDDS_purchasedBadSeason', 'TVA_USD_PPP_pmae_pday',
       'currency_conversion_factor', 'total_income_USD_PPP_pHH_Yr',
       'offfarm_income_USD_PPP_pHH_Yr', 'farm_income_USD_PPP_pHH_Yr',
       'value_farm_produce_USD_PPP_pHH_Yr', 'crop_sales_USD_PPP_pHH_Yr',
       'value_crop_produce_USD_PPP_pHH_Yr',
       'value_crop_consumed_USD_PPP_pHH_Yr',
       'livestock_prodsales_USD_PPP_pHH_Yr',
       'value_livestock_production_USD_PPP_pHH_Yr',
       'value_livestock_prod_consumed_USD_PPP_pHH_Yr', 'Market_Orientation',
       'Livestock_Orientation', 'Food_Availability

In [48]:
min_values = set_min_value(data_pre)
max_values = set_max_value(data_pre)

In [49]:
print(min_values)

[-inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, 1, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf]


In [50]:
imp = IterativeImputer(max_iter=10, random_state=0, min_value=min_values, max_value=max_values)
data = data_pre.drop(['RHoMIS_ID','ID_HH'],axis=1)
# imp.fit(data)
# data = imp.transform(data)
data_imputed = pd.DataFrame(imp.fit_transform(data),columns = data.columns)

In [60]:
data_imputed['Food_InsecurityLevel'] = data_imputed['Food_InsecurityLevel'].astype('int32')

In [63]:
data_imputed.to_pickle('imputed_data.pkl')

In [62]:
print(data_imputed['Food_InsecurityLevel'].unique())

[3 4 2 1]


In [52]:
y = data_imputed['Food_InsecurityLevel']
x = data_imputed.drop(['Food_InsecurityLevel'],axis=1)
# from sklearn.linear_model import LinearRegression
# reg = LinearRegression().fit(x, y)